# Introduction
This dataset represents Bulk tissue expression RNA-Seq samples from GTEx Analysis V10 release.


The input files were found at [https://www.gtexportal.org/home/downloads/adult-gtex/bulk_tissue_expression]

# Libraries
Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
print(ad.__version__)
from tqdm import tqdm

0.11.4


# Import Gene/Transcript TPMs 
Import the gene expression data from gTEX. 

```bash
# Gene Expression Transcripts Per Million (TPM) from RNASeQCv2.4.2.
pixi run wget -P gtex_bulk_tissue_expression https://storage.googleapis.com/adult-gtex/bulk-gex/v10/rna-seq/GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.gct.gz
```

* The first two columns contain the gene/transcript IDs and a description
* The values are a matrix of 19616 tissue sample IDs as named in each column by 59033 genes as named in each row from RNASeQCv2.4.2.

In [2]:

genes_f = "gtex_bulk_tissue_expression/GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.gct.gz"

first get the gene/transcript meta data from the quantification files

In [3]:
# nrows = 1000
nrows = None
chunkSize = 1000
if nrows:
    genes_meta_df = pd.read_csv(
        genes_f, 
        sep="\t", skiprows=2, nrows=nrows,
        usecols=[0,1], compression='gzip')
else:
    total_iterations = np.ceil(59036 / chunkSize)
    genes_meta_df = pd.concat(
        [chunk for chunk in tqdm(pd.read_csv(genes_f, sep="\t", skiprows=2, usecols=[0,1], compression='gzip', chunksize=chunkSize), desc='Loading data', total=total_iterations)]
    )
genes_meta_df.set_index('Name', inplace=True)

Loading data: 100%|██████████| 60/60.0 [00:24<00:00,  2.48it/s]


then get the TPMs from the quantification files

In [4]:

if nrows:
    genes_counts_df = pd.read_csv(
        genes_f, 
        sep="\t", skiprows=2, nrows=nrows,
        usecols=range(2,2+19616), compression='gzip', dtype=np.float64)
else:
    genes_counts_df = pd.concat(
        [chunk for chunk in tqdm(pd.read_csv(genes_f, sep="\t", skiprows=2, usecols=range(2,2+19616), compression='gzip', dtype=np.float64, chunksize=chunkSize), desc='Loading data', total=total_iterations)]
    )

Loading data: 100%|██████████| 60/60.0 [02:21<00:00,  2.36s/it]


In [5]:
genes_counts_df.loc[:,"GTEX-1117F-0226-SM-5GZZ7"].head()

0    0.000000
1    9.678340
2    0.000000
3    0.073473
4    0.000000
Name: GTEX-1117F-0226-SM-5GZZ7, dtype: float64

# Import meta data for the gene/transcripts

In [6]:
def parse_gtf_attributes(attr_string):
    attributes = {}
    for item in attr_string.strip().split(';'):
        if item.strip():
            parts = item.strip().split(' ', 1) # Split only on the first space
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip().strip('"') # Remove quotes
                attributes[key] = value
    return attributes
parse_gtf_attributes('gene_id "ENSG00000223972.5"; transcript_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"')

{'gene_id': 'ENSG00000223972.5',
 'transcript_id': 'ENSG00000223972.5',
 'gene_type': 'transcribed_unprocessed_pseudogene'}

In [7]:
#from gtfparse import read_gtf

# returns GTF with essential columns such as "feature", "seqname", "start", "end"
# alongside the names of any optional keys which appeared in the attribute column
gtf_df = pd.read_csv("gtex_reference/gencode.v39.GRCh38.genes.gtf", sep="\t", comment="#", header=None, dtype=str)
gtf_df.columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
gene_gtf_df = gtf_df[gtf_df["feature"] == "gene"].copy()
add_columns = ['gene_id', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name']
gene_gtf_df.loc[:, add_columns] = None

for index, row in gene_gtf_df.iterrows():
    attributes_dict = parse_gtf_attributes(row['attribute'])
    #gene_gtf_df.at[index,'attribute'] = attributes_dict
    for col in add_columns:
        if col in attributes_dict.keys():
            gene_gtf_df.loc[index,col] = attributes_dict[col]
genes_meta_df.index.name =None
gene_gtf_df = gene_gtf_df.set_index('gene_id').copy()
gene_gtf_df.index.name =None
gene_merged_meta_df = genes_meta_df.merge(gene_gtf_df, left_index=True, right_index=True, how='left')
print(f"Number of genes in metadata: {len(genes_meta_df)}")
print(f"Number of genes in gtf: {len(gene_gtf_df)}")
gene_merged_meta_df.head()


Number of genes in metadata: 59033
Number of genes in gtf: 59033


,Description,seqname,source,feature,start,end,score,strand,frame,attribute,transcript_id,gene_type,gene_name,transcript_type,transcript_name
ENSG00000223972.5,DDX11L1,chr1,HAVANA,gene,11869,14403,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN...",ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,transcribed_unprocessed_pseudogene,DDX11L1
ENSG00000227232.5,WASH7P,chr1,HAVANA,gene,14410,29553,.,-,.,"gene_id ""ENSG00000227232.5""; transcript_id ""EN...",ENSG00000227232.5,unprocessed_pseudogene,WASH7P,unprocessed_pseudogene,WASH7P
ENSG00000278267.1,MIR6859-1,chr1,ENSEMBL,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267.1""; transcript_id ""EN...",ENSG00000278267.1,miRNA,MIR6859-1,miRNA,MIR6859-1
ENSG00000243485.5,MIR1302-2HG,chr1,HAVANA,gene,29571,31109,.,+,.,"gene_id ""ENSG00000243485.5""; transcript_id ""EN...",ENSG00000243485.5,lncRNA,MIR1302-2HG,lncRNA,MIR1302-2HG
ENSG00000237613.2,FAM138A,chr1,HAVANA,gene,34554,36081,.,-,.,"gene_id ""ENSG00000237613.2""; transcript_id ""EN...",ENSG00000237613.2,lncRNA,FAM138A,lncRNA,FAM138A


# Import meta data for the samples

In [8]:
sample_meta_f = "gtex_metadata/GTEx_Analysis_v10_Annotations_SampleAttributesDS.txt"
total_iterations = np.ceil(48232 / chunkSize)
sample_meta_df = pd.concat(
    [chunk for chunk in tqdm(pd.read_csv(sample_meta_f, sep="\t", chunksize=chunkSize), desc='Loading data', total=total_iterations)]
)

Loading data: 100%|██████████| 49/49.0 [00:00<00:00, 205.19it/s]


limit sample meta information to the samples used for the RNA-seq and relavent columns

I used excel to open `GTEx_Analysis_v10_Annotations_SampleAttributesDD.xlsx` to distinuish the columns of interest. The columns I used are:
* SAMPID: Sample ID, GTEx Public Sample ID
* SMTS: Tissue Type, area from which the tissue sample was taken. This is a parent value to SMTSD. (ie. Adipose Tissue, Adrenal Gland, Bladder, Blood, Blood Vessel, Bone Marrow, Brain, Breast, Cervix Uteri, Colon, Esophagus, Fallopian Tube, Heart, Kidney, Liver, Lung, Muscle, Nerve, Ovary, Pancreas, Pituitary, Prostate, Skin, Spleen, Stomach, Testis, Thyroid, Uterus, Vagina)
* SMTSD: Tissue Type, more specific detail of tissue type


In [9]:
rnaseq_samp_meta_df = sample_meta_df.loc[sample_meta_df["SMAFRZE"]=="RNASEQ",["SAMPID", "SMTS", "SMTSD"]].copy()
rnaseq_samp_meta_df = (
    rnaseq_samp_meta_df.rename(
        columns={
            "SMTS": "TISSUE",
            "SMTSD": "TISSUE_DETAILED"
        }
    )
)
rnaseq_samp_meta_df.set_index("SAMPID", inplace=True)
rnaseq_samp_meta_df = rnaseq_samp_meta_df.loc[genes_counts_df.columns.tolist(),:]
print(rnaseq_samp_meta_df.shape)
rnaseq_samp_meta_df.head()

(19616, 2)


,TISSUE,TISSUE_DETAILED
SAMPID,,
GTEX-1117F-0005-SM-HL9SH,Blood,Whole Blood
GTEX-1117F-0011-R10b-SM-GI4VE,Brain,Brain - Frontal Cortex (BA9)
GTEX-1117F-0011-R11b-SM-GIN8R,Brain,Brain - Cerebellar Hemisphere
GTEX-1117F-0011-R2b-SM-GI4VL,Brain,Brain - Substantia nigra
GTEX-1117F-0011-R3a-SM-GJ3PJ,Brain,Brain - Anterior cingulate cortex (BA24)


# Initializing AnnData
See the following tutorial at [https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html]


Build an AnnData object with gene expression counts

In [10]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg"
      , width=300, height=300)

In [11]:
counts = csr_matrix(
    genes_counts_df.transpose(), 
    dtype=np.float32)
adata = ad.AnnData(counts)
adata

AnnData object with n_obs × n_vars = 19616 × 59033

We see that AnnData provides a representation with summary statistics of the data. The initial data we passed are accessible as a sparse matrix using adata.X

In [12]:
adata.X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 610491479 stored elements and shape (19616, 59033)>

Now, we provide the index to both the `obs` and `var` axes using `.obs_names` (resp. `.var_names`).

* `adata.var_names` = transcript or gene names
* `adata.obs_names` = sample names

In [13]:
adata.var_names = genes_meta_df.index.tolist()
adata.obs_names = genes_counts_df.columns.tolist()
#print(adata.var_names[0:10])
print(adata.var_names[-10:-1])
#print(adata.obs_names[0:10])
print(adata.obs_names[-10:-1])

Index(['ENSG00000198886.2', 'ENSG00000210176.1', 'ENSG00000210184.1',
       'ENSG00000210191.1', 'ENSG00000198786.2', 'ENSG00000198695.2',
       'ENSG00000210194.1', 'ENSG00000198727.2', 'ENSG00000210195.2'],
      dtype='object')
Index(['GTEX-ZZPU-1326-SM-5GZWS', 'GTEX-ZZPU-1426-SM-5GZZ6',
       'GTEX-ZZPU-1826-SM-5E43L', 'GTEX-ZZPU-2126-SM-5EGIU',
       'GTEX-ZZPU-2226-SM-5EGIV', 'GTEX-ZZPU-2326-SM-GOQYU',
       'GTEX-ZZPU-2426-SM-5E44I', 'GTEX-ZZPU-2526-SM-GOQZ3',
       'GTEX-ZZPU-2626-SM-5E45Y'],
      dtype='object')


## Subsetting AnnData

These index values can be used to subset the AnnData, which provides a view of the AnnData object. We can imagine this to be useful to subset the AnnData to particular cell types or gene modules of interest. The rules for subsetting AnnData are quite similar to that of a Pandas DataFrame. You can use values in the `obs/var_names`, boolean masks, or cell index integers.

In [14]:
adata[['GTEX-1117F-0226-SM-5GZZ7', 'GTEX-1117F-0011-R10b-SM-GI4VE'], ['ENSG00000223972.5', 'ENSG00000227232.5']]

View of AnnData object with n_obs × n_vars = 2 × 2

# Adding aligned metadata

## Observation/Variable level

So we have the core of our object and now we’d like to add metadata at both the observation and variable levels. This is pretty simple with AnnData, both `adata.obs` and `adata.var` are Pandas DataFrames.

In [15]:
## Categoricals are preferred for efficiency
adata.obs["TISSUE"] = pd.Categorical(rnaseq_samp_meta_df['TISSUE'])  
adata.obs["TISSUE_DETAILED"] = pd.Categorical(rnaseq_samp_meta_df['TISSUE_DETAILED'])  
adata.obs

,TISSUE,TISSUE_DETAILED
GTEX-1117F-0005-SM-HL9SH,Blood,Whole Blood
GTEX-1117F-0011-R10b-SM-GI4VE,Brain,Brain - Frontal Cortex (BA9)
GTEX-1117F-0011-R11b-SM-GIN8R,Brain,Brain - Cerebellar Hemisphere
GTEX-1117F-0011-R2b-SM-GI4VL,Brain,Brain - Substantia nigra
GTEX-1117F-0011-R3a-SM-GJ3PJ,Brain,Brain - Anterior cingulate cortex (BA24)
...,...,...
GTEX-ZZPU-2326-SM-GOQYU,Nerve,Nerve - Tibial
GTEX-ZZPU-2426-SM-5E44I,Blood Vessel,Artery - Tibial
GTEX-ZZPU-2526-SM-GOQZ3,Skin,Skin - Sun Exposed (Lower leg)
GTEX-ZZPU-2626-SM-5E45Y,Muscle,Muscle - Skeletal


In [16]:
#if run_transcripts:
#    adata.var["gene_id"] = gene_merged_meta_df['gene_id']
#else:
#    adata.var["Description"] = gene_merged_meta_df['Description']
#    adata.var["transcript_id"]
adata.var = gene_merged_meta_df.copy()

We can also see now that the AnnData representation has been updated:

In [17]:
adata

AnnData object with n_obs × n_vars = 19616 × 59033
    obs: 'TISSUE', 'TISSUE_DETAILED'
    var: 'Description', 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name'

## Subsetting using metadata

We can also subset the AnnData using `.obs` and `.var` attributes, which are Pandas DataFrames. This is useful to filter the AnnData object to only those samples or genes that are of interest. For example, we can filter the AnnData to only those samples that have a particular tissue type:

In [18]:
adata[adata.obs.TISSUE == "Adipose Tissue", :]

View of AnnData object with n_obs × n_vars = 1301 × 59033
    obs: 'TISSUE', 'TISSUE_DETAILED'
    var: 'Description', 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name'

We can also subset using the transcript or gene ID meta data. For example, we can filter the AnnData to only those genes that are on chromosome 12:

In [19]:
adata[:,adata.var.seqname == "chr12"]

View of AnnData object with n_obs × n_vars = 19616 × 2953
    obs: 'TISSUE', 'TISSUE_DETAILED'
    var: 'Description', 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name'

# Observation/variable-level matrices
We might also have metadata at either level that has many dimensions to it, such as a UMAP embedding of the data. For this type of metadata, AnnData has the `.obsm/.varm` attributes. We use keys to identify the different matrices we insert. The restriction of `.obsm/.varm` are that `.obsm` matrices must length equal to the number of observations as `.n_obs` and `.varm` matrices must length equal to .n_vars. They can each independently have different number of dimensions.

In the code below we make a randomly generated matrix that we can interpret as a UMAP embedding of the data we’d like to store, as well as some random gene-level metadata. However, this code is within a False conditional so it will not run because we do not have any UMAP embeddings or gene-level metadata to add to the AnnData object.

In [20]:
if 1==0:
    # for each cell add that it has 2 UMAP coordinates in the table "X_umap"
    # in `adata.obsm` (obsm = obs matrix)
    adata.obsm["X_umap"] = np.random.normal(0, 1, size=(adata.n_obs, 2)) 

    # for each transcript add 5 features of interest in the table "gene_stuff"
    # in `adata.varm` (varm = var matrix)
    #adata.varm["gene_stuff"] = np.random.normal(0, 1, size=(adata.n_vars, 5))
    gene_lengths_df = pd.read_csv("salmon.merged.gene_lengths.tsv", sep="\t", index_col=0) 
    adata.varm["gene_lengths"] = gene_lengths_df.loc[:,sample_sheet_df['sample'].tolist()]
    adata.obsm

Again, the AnnData representation is updated.

In [21]:
adata

AnnData object with n_obs × n_vars = 19616 × 59033
    obs: 'TISSUE', 'TISSUE_DETAILED'
    var: 'Description', 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name'

A few more notes about .obsm/.varm

The “array-like” metadata can originate from a Pandas DataFrame, scipy sparse matrix, or numpy dense array.

When using scanpy, their values (columns) are not easily plotted, where instead items from .obs are easily plotted on, e.g., UMAP plots.



# Unstructured metadata
AnnData has .uns, which allows for any unstructured metadata. This can be anything, like a list or a dictionary with some general information that was useful in the analysis of our data.

In [22]:
adata.uns["Release"] = "GTEx v10"
adata.uns["Type"] = "RNASeq Bulk Tissue Expression"
adata.uns["Name"] = genes_f
adata.uns["Description"] = "Gene Expression Transcripts Per Million (TPM) from RNASeQCv2.4.2."

# Layers
Finally, we may have different forms of our original core data, perhaps one that is normalized and one that is not. These can be stored in different layers in AnnData. For example, let’s log transform the original data and store it in a layer:

In [23]:
adata.layers["log_transformed"] = np.log1p(adata.X)
adata

AnnData object with n_obs × n_vars = 19616 × 59033
    obs: 'TISSUE', 'TISSUE_DETAILED'
    var: 'Description', 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name'
    uns: 'Release', 'Type', 'Name', 'Description'
    layers: 'log_transformed'

We may also want to have layers for the gene read counts by tissue.

In [24]:
if 1==0:
    gene_tpm_df = pd.read_csv("salmon.merged.gene_tpm.tsv", sep="\t", index_col=0) 
    adata.layers["gene_tpm"] = csr_matrix(
        gene_tpm_df.loc[:,sample_sheet_df['sample'].tolist()].transpose(), 
        dtype=np.float32)

# Conversion to DataFrames
We can also ask AnnData to return us a DataFrame from one of the layers:

In [25]:
adata.to_df(layer="log_transformed")

,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,ENSG00000186092.7,ENSG00000238009.6,ENSG00000233750.3,...,ENSG00000198886.2,ENSG00000210176.1,ENSG00000210184.1,ENSG00000210191.1,ENSG00000198786.2,ENSG00000198695.2,ENSG00000210194.1,ENSG00000198727.2,ENSG00000210195.2,ENSG00000210196.2
GTEX-1117F-0005-SM-HL9SH,0.000000,0.847339,0.0,0.000000,0.000000,0.000000,0.022926,0.000000,0.031386,0.037188,...,6.737683,0.000000,0.000000,0.000000,4.639427,4.572170,0.000000,6.067395,0.000000,0.000000
GTEX-1117F-0011-R10b-SM-GI4VE,0.000000,1.521542,0.0,0.089681,0.000000,0.000000,0.028326,0.045503,0.038741,0.057018,...,10.815031,0.778210,1.322910,0.000000,8.897326,8.474194,2.011786,10.263868,1.405545,0.786132
GTEX-1117F-0011-R11b-SM-GIN8R,0.000000,2.414958,0.0,0.033619,0.000000,0.068124,0.060926,0.076191,0.014291,0.016960,...,11.073470,1.146005,0.000000,1.558837,9.218863,8.879225,2.899486,10.560985,1.702245,2.128507
GTEX-1117F-0011-R2b-SM-GI4VL,0.000000,1.377884,0.0,0.000000,0.017506,0.000000,0.044661,0.063710,0.000000,0.000000,...,11.012090,1.415831,1.038278,1.316092,9.279017,8.969584,2.790398,10.505116,2.518986,1.992190
GTEX-1117F-0011-R3a-SM-GJ3PJ,0.000000,1.543896,0.0,0.000000,0.000000,0.000000,0.060280,0.033008,0.000000,0.016777,...,10.841954,0.821233,0.689513,1.119196,8.869345,8.636981,2.588410,10.185699,1.168959,1.819392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTEX-ZZPU-2326-SM-GOQYU,0.000000,1.218987,0.0,0.040252,0.000000,0.000000,0.024844,0.039950,0.098713,0.088438,...,10.381987,0.415739,0.000000,0.406112,8.991590,9.087047,3.585001,10.101030,0.431083,1.128912
GTEX-ZZPU-2426-SM-5E44I,0.000000,1.357406,0.0,0.051950,0.000000,0.000000,0.063271,0.116421,0.184021,0.051979,...,9.739143,0.512368,0.578112,0.000000,9.138722,9.708737,3.909109,9.882345,0.000000,1.312737
GTEX-ZZPU-2526-SM-GOQZ3,0.017635,1.807602,0.0,0.000000,0.000000,0.104232,0.160444,0.011681,0.139361,0.085123,...,10.216205,0.368662,0.419604,0.359939,9.231632,9.371932,3.716541,10.141315,1.812655,1.032978
GTEX-ZZPU-2626-SM-5E45Y,0.020019,0.611104,0.0,0.000000,0.000000,0.040779,0.024425,0.204595,0.016854,0.000000,...,10.624352,0.409922,1.215011,0.000000,9.628077,9.882213,4.020404,10.517957,1.655911,1.631827


We see that the `.obs_names/.var_names` are used in the creation of this Pandas object.

# Writing the results to disk
`AnnData` comes with its own persistent HDF5-based file format: h5ad. If string columns with small number of categories aren’t yet categoricals, `AnnData` will auto-transform to categoricals.

In [ ]:
adata.write('GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.h5ad', compression="gzip")

In [29]:
!h5ls 'GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.h5ad'

X                        Group
layers                   Group
obs                      Group
obsm                     Group
obsp                     Group
uns                      Group
var                      Group
varm                     Group
varp                     Group


In [28]:
import zarr
adata.write_zarr('GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.zarr')
